# 使用少量样本示例 How to use few shot examples

在本指南中，我们将学习如何创建一个简易的提示模板，该模板在生成内容时向模型提供输入和输出示例。  
向LLM（大型语言模型）提供几个这样的示例称为少量样本引导（few-shotting），这是一种简单而强大的方法，可以指导内容生成，并在某些情况下显著提高模型性能。

少量样本引导的提示模板可以从一组示例或从负责从定义集选择示例子集的示例选择器类构建。

本指南将涵盖使用字符串提示模板进行少量样本引导的内容。

## 为少样本示例创建格式化程序
配置一个格式化程序，将少样本示例格式化为字符串。此格式化程序应为 PromptTemplate 对象。

In [2]:
from langchain_core.prompts import PromptTemplate

example_prompt = PromptTemplate.from_template("问题:{question}\n{answer}")

## 创建示例集 

接下来，我们将创建一个少样本示例列表。每个示例都应该是一个字典，表示我们上面定义的格式化程序提示的示例输入。

In [7]:
examples = [
	{
		"question": "爱因斯坦和霍金谁的寿命更长？",
		"answer": """
在这里需要进一步的问题来确认：是。
后续问题：爱因斯坦去世时多大？
中间答案：爱因斯坦去世时76岁。
后续问题：霍金去世时多大？
中间答案：霍金去世时76岁。
最终答案是：相同""",
	},
	{
		"question": "微博的创始人是哪一年出生的？",
		"answer": """
在这里需要进一步的问题来确认：是。
后续问题：微博的创始人是谁？
中间答案：微博的创始人是王高飞。
后续问题：王高飞是哪一年出生的？
中间答案：王高飞出生于1978年。
最终答案是：1978年""",
	},
	{
		"question": "《红楼梦》和《水浒传》的作者是同一个人吗？",
		"answer": """
在这里需要进一步的问题来确认：是。
后续问题：《红楼梦》的作者是谁？
中间答案：《红楼梦》的作者是曹雪芹。
后续问题：《水浒传》的作者是谁？
中间答案：《水浒传》的作者是施耐庵。
最终答案是：不是""",
	},
	{
		"question": "《大白鲨》和《007：大战皇家赌场》的导演都来自同一个国家吗？",
		"answer": """
在这里需要进一步的问题来确认。
跟进问题：《大白鲨》的导演是谁？
中间答案：《大白鲨》的导演是史蒂文·斯皮尔伯格。
跟进问题：史蒂文·斯皮尔伯格来自哪里？
中间答案：美国。
跟进问题：《007：大战皇家赌场》的导演是谁？
中间答案：《007：大战皇家赌场》的导演是马丁·坎贝尔。
跟进问题：马丁·坎贝尔来自哪里？
中间答案：新西兰。
所以最终答案是：不，他们并非来自同一个国家。""",
	}										
]

让我们用其中一个例子来测试格式提示：

In [8]:
print(example_prompt.invoke(examples[0]).to_string())

问题:爱因斯坦和霍金谁的寿命更长？

在这里需要进一步的问题来确认：是。
后续问题：爱因斯坦去世时多大？
中间答案：爱因斯坦去世时76岁。
后续问题：霍金去世时多大？
中间答案：霍金去世时76岁。
最终答案是：相同


## 将示例和格式化程序传递给 FewShotPromptTemplate 

最后，创建一个FewShotPromptTemplate对象。这个对象接收少量样本示例和用于处理这些少量样本的格式化器。  
当这个FewShotPromptTemplate被格式化时，它会使用example_prompt格式化传入的示例，然后将它们添加到最终提示中，在后缀之前：

In [11]:
from langchain_core.prompts import FewShotPromptTemplate

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:{input}",
    input_variables=["input"]
)

print(prompt.invoke({"input":"红楼梦的作者是谁？"}).to_string())

问题:爱因斯坦和霍金谁的寿命更长？

在这里需要进一步的问题来确认：是。
后续问题：爱因斯坦去世时多大？
中间答案：爱因斯坦去世时76岁。
后续问题：霍金去世时多大？
中间答案：霍金去世时76岁。
最终答案是：相同

问题:微博的创始人是哪一年出生的？

在这里需要进一步的问题来确认：是。
后续问题：微博的创始人是谁？
中间答案：微博的创始人是王高飞。
后续问题：王高飞是哪一年出生的？
中间答案：王高飞出生于1978年。
最终答案是：1978年

问题:《红楼梦》和《水浒传》的作者是同一个人吗？

在这里需要进一步的问题来确认：是。
后续问题：《红楼梦》的作者是谁？
中间答案：《红楼梦》的作者是曹雪芹。
后续问题：《水浒传》的作者是谁？
中间答案：《水浒传》的作者是施耐庵。
最终答案是：不是

问题:《大白鲨》和《007：大战皇家赌场》的导演都来自同一个国家吗？

在这里需要进一步的问题来确认。
跟进问题：《大白鲨》的导演是谁？
中间答案：《大白鲨》的导演是史蒂文·斯皮尔伯格。
跟进问题：史蒂文·斯皮尔伯格来自哪里？
中间答案：美国。
跟进问题：《007：大战皇家赌场》的导演是谁？
中间答案：《007：大战皇家赌场》的导演是马丁·坎贝尔。
跟进问题：马丁·坎贝尔来自哪里？
中间答案：新西兰。
所以最终答案是：不，他们并非来自同一个国家。

Question:红楼梦的作者是谁？


## 使用示例选择器
我们将重用上一步中的示例集和格式化程序。  
但是，我们不会将示例直接输入到 FewShotPromptTemplate 对象中，而是将它们输入到名为 SemanticSimilarityExampleSelector 实例的 ExampleSelector 实现中。  
此类根据与输入的相似性从初始集合中选择少量示例。  
它使用嵌入模型来计算输入与少量示例之间的相似性，并使用向量存储来执行最近邻搜索。

为了展示它的样子，让我们初始化一个实例并单独调用它：

In [12]:
import os
from dotenv import load_dotenv,find_dotenv

_ = load_dotenv(find_dotenv())

In [20]:
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_community.embeddings import BaichuanTextEmbeddings

embeddings = BaichuanTextEmbeddings(baichuan_api_key=os.environ["BAICHUAN_API_KEY"])

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # 这是可供选择的示例列表。
    examples,
    # 这是用于生成用于测量语义相似度的嵌入的嵌入类。
    embeddings,
    # 这是用于存储嵌入并进行相似性搜索的 VectorStore 类。
    Chroma,
    # 这是要生成的示例的数量。
    k=1
)

# 选择与输入最相似的例子。
question = "红楼梦的作者是谁?"
selected_examples = example_selector.select_examples({"question":question})
print(f"Examples most similar to the input : {question}")
for example in selected_examples:
    print("\n")
    for k,v in example.items():
        print(f"{k} : {v}")

Examples most similar to the input : 红楼梦的作者是谁?


answer : 
在这里需要进一步的问题来确认：是。
后续问题：《红楼梦》的作者是谁？
中间答案：《红楼梦》的作者是曹雪芹。
后续问题：《水浒传》的作者是谁？
中间答案：《水浒传》的作者是施耐庵。
最终答案是：不是
question : 《红楼梦》和《水浒传》的作者是同一个人吗？


现在，让我们创建一个 FewShotPromptTemplate 对象。  
此对象接收示例选择器和少样本示例的格式化程序提示。

In [21]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"]
)

print(prompt.invoke({"input":"大白鲨的导演是谁？"}).to_string())

问题:《大白鲨》和《007：大战皇家赌场》的导演都来自同一个国家吗？

在这里需要进一步的问题来确认。
跟进问题：《大白鲨》的导演是谁？
中间答案：《大白鲨》的导演是史蒂文·斯皮尔伯格。
跟进问题：史蒂文·斯皮尔伯格来自哪里？
中间答案：美国。
跟进问题：《007：大战皇家赌场》的导演是谁？
中间答案：《007：大战皇家赌场》的导演是马丁·坎贝尔。
跟进问题：马丁·坎贝尔来自哪里？
中间答案：新西兰。
所以最终答案是：不，他们并非来自同一个国家。

Question: 大白鲨的导演是谁？
